# Lesson Plan

In this notebook, we're going to look at variational autoencoders and denoising autoencoders. I've designed this workbook as an example of what can be used as a workbook for a session on VAEs and denoising autoencoders.

Before we get started with variational autoencoders and denoising autoencoders, let's review regular autoencoders. An autoencoder is a type of artificial neural network that is used in representation learning. That means it can be used to learn a compact representation of an input data set, called an encoding, through the use of unsupervised learning. The autoencoder is trained to reconstruct the original input data from this encoding. An autoencoder consists of two main components: the encoder and the decoder. The encoder processes the input data and learns a compressed representation of it, called the encoding, by minimizing the reconstruction loss, which is the difference between the input data and the reconstructed data. The decoder processes the encoding and generates a reconstruction of the original input data. Here's a simple visual that helps to understand what's going on:

![Autoencoder Image](autoencoder.jpg)

Now that we know what the basic autoencoder architecture looks like, we can move onto VAEs and denoising autoencoders. Do the folloing readings by focusing on the description:

[Aurélien Géron (2019) Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition. Chapter 17](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

- Focus on page 586-591. You should aim to understand what makes VAEs probabilistic and generative models. Make sure you also understand why we add the latent loss term, and why the latent loss equal to minimizing the KL divergnece between the Standard Normal distribution and the target distribution.

[Optional] [DigitalSreeni (Nov 26, 2020), 178 - An introduction to variational autoencoders (VAE)](https://www.youtube.com/watch?v=YV9D3TWY5Zo)

- If you're a visual learner, this video can complement what is in the readings. It also explains how the training works by explaining how we use KL divergence to minimize the distance between the Standard Normal distribution and the target distribution.

[Optional] [Kingma, D. P., & Welling, M. (2013). Auto-Encoding Variational Bayes](https://doi.org/10.48550/arXiv.1312.6114)

- If you want to read more on VAEs, this is the original paper that introduced the idea. It discusses the idea that we can use Variational Bayesian Inference to approximate the posterior and create the Normal distribution, which we can then sample from to pass to our decoder.


[Aurélien Géron (2019) Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition. Chapter 17](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

- For denoising autoencoders, focus on page 581-582. Make sure you understand the different ways we can construct denoising autoencoders and how they build upon regular autoencoders

[Optional] [Developers Hutt (Jan 10, 2021), Denoising AutoEnocoder || Autoencoders || Developers Hutt](https://www.youtube.com/watch?v=ebPq0cILZV8)

- This video is also a good complement to the reading above

[Optional] [Vincent, P., Larochelle, H., Lajoie, I., Bengio, Y., & Manzagol, P.-A. (2010). Stacked Denoising Autoencoders: Learning Useful Representations in a Deep Network with a Local Denoising Criterion. Journal of Machine Learning Research, 11(110), 3371–3408.](http://jmlr.org/papers/v11/vincent10a.html)

- This is the original paper on denoising autoencoders if you want to read further

[Google Cloud Tech (Aug 14, 2018), Getting Started with Keras](https://www.youtube.com/watch?v=J6Ok8p463C4&t=317s)

- This short video will help you get started with the keras API using TensorFlow. This will be helpful for the implementation we'll do in the workbook below

## 2 - Basic Questions

1. Intuitively explain what VAEs and denoising autoencoders are and what makes them different from regular autoencoders.
2. What are the different ways we can create denoising autoencoders?

Your answer here

## 3 - Core Questions

The code cell below creates the encoder network for a VAE. Explain what each line is doing. Specifically, explain the following:
- How many layers do we have in the encoder?
- How many neurons do we have in each layer?
- What is the size of the image that the encoder accepts and what is its output?

In [3]:
# import the necessary libraries

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
codings_size = 10

inputs = keras.layers.Input(shape=[28, 28])
z = keras.layers.Flatten()(inputs)
z = keras.layers.Dense(150, activation="selu")(z)
z = keras.layers.Dense(100, activation="selu")(z)
codings_mean = keras.layers.Dense(codings_size)(z) 
codings_log_var = keras.layers.Dense(codings_size)(z) 
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = keras.Model(inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])

The code cell below contains that code for creating a denoising autoencoder by adding Gaussian noise to the input. Use the same outline to create one that instead uses dropout

In [ ]:
# Denoising Autoencoder by adding a Gaussian Noise

codings_size = 10  

# create the encoder with two hidden layers
denoising_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.GaussianNoise(0.2),  # gaussian noise added
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(codings_size, activation="selu")
])

# create the decoder with two hidden layers
denoising_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[codings_size]),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

denoising_ae = keras.models.Sequential([denoising_encoder, denoising_decoder])  # connect the decoder and encoder
denoising_ae.compile(loss="binary_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=1.0))

In [1]:
# Create the denoising autoencoder with dropout here

## 4 - Extension Questions

We said that for VAEs, we can sample from the Normal distribution created to then pass on to the decoder network to get an output. The following code can sample from the Normal distribution

In [4]:
K = keras.backend

# Helps with sampling from the Normal distribution created by the encoder. We then feed this to the decoder
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean

We can see that we are using mean and log_var instead of regular mean and variance for our Normal distribution. Why are we taking the log of the variance? What problem does that solve?

Your answer here

In all the autoencoders we've used so far, we used feedforward neural networks in both our encoder and decoder networks. But since we're working with images, convolutional neural networks might do a better job. Therefore, in the code cell below, create a variational autoencoder that uses convolutional neural networks in the encoder and decoder networks

In [5]:
# Your work here